In [35]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [36]:
data = pd.read_csv('train.csv')

In [37]:
data.head()


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,78,1,1,3,2.312,4
1,42,0,2,2,3.560,3
2,31,0,1,3,2.926,3
3,34,1,2,3,1.924,3
4,27,1,2,3,1.807,3


In [38]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('Drug', axis=1), data['Drug'], test_size=0.2, random_state=0)

In [39]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.485625

In [40]:
#logistic regression algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

lr_clf.score(x_val, y_val)

C:\Users\Ianme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8875

In [41]:
#import min max scaler, one hot encoder, and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['BP', 'Cholesterol', 'Na_to_K']
numerical_features = ['BP', 'Cholesterol', 'Na_to_K']
categorical_features = []

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [42]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV

# define the parameters
params = {
    'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'tol' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'C' : [1.0, 2.0, 3.0, 4.0],
    'penalty' : ['l1', 'l2', 'elasticnet'],
    'dual' : [0, 1],
}
# create the model
lr_clf = LogisticRegression()

# create the random search cv
random_search = RandomizedSearchCV(lr_clf, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = -1, cv = 5, verbose = 3)

# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


ValueError: 
All the 25 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Ianme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Ianme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Ianme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.p

In [43]:
# train lr with the best params
lr_clf = LogisticRegression(**random_search.best_params_)
lr_clf.fit(x_train, y_train)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [44]:
lr_clf.score(x_val, y_val)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [45]:
data_test = pd.read_csv('test.csv')

In [47]:
data_test['Drug'].shape

(2000,)

In [49]:
x_test = preprocessor.transform(data_test)
y_test = lr_clf.predict(x_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['Drug'], y_test)

NameError: name 'y_test' is not defined